<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 4 : Visualisation de vecteurs de mots

## R√©sum√©
Le but de ce laboratoire est de visualiser des vecteurs de mots gr√¢ce √† des m√©thodes de r√©duction de dimensionnalit√©.  La visualisation permettra de deviner plus facilement le "mot du jour" dans le jeu en ligne [C√©mantix](https://cemantix.certitudes.org/).

## 1. Prise en main de word2vec

Le mod√®le Word2vec contient des _embeddings_ des mots qui sont appris automatiquement √† partir de textes.  Chaque mot est donc repr√©sent√© par un vecteur dans un espace avec plusieurs centaines de dimensions (p.ex. 300 ou 500).  Dans cet espace, les mots de sens ou d'usage proches ont des vecteurs proches (les _embeddings_ capturent la similarit√© entre mots).  Mais la dimensionnalit√© de l'espace fait qu'il est difficile de visualiser les vecteurs de mots.

La librairie [Gensim](https://radimrehurek.com/gensim/index.html) permet de charger un mod√®le Word2vec existant (avec la fonction `load_word2vec_format(...)`) et de manipuler les vecteurs de mots (instances de la classe `KeyedVectors`).  On peut notamment utiliser la fonction `similarity(w1, w2)` qui retourne le cosinus des vecteurs correspondant aux mots `w1` et `w2`, √† condition que ceux-ci soient connus du mod√®le.  On peut aussi utiliser la fonction `most_similar([w])` qui retourne les mots voisins d'un ou plusieurs mots.  La [documentation de KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) fournit des exemples utiles.

Veuillez t√©l√©charger l'_un des deux_ mod√®les word2vec suivants, d√©j√† entra√Æn√©s, mis √† disposition par [J.-Ph. Fauconnier](https://fauconnier.github.io/#data):
- [frWac_no_postag_no_phrase_500_cbow_cut100.bin](https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin) - plus petit (229 Mo), mais pas identique au mod√®le utilis√© pour C√©mantix ;
- [frWac_no_postag_phrase_500_cbow_cut10.bin](https://embeddings.net/embeddings/frWac_no_postag_phrase_500_cbow_cut10.bin) - bien plus grand (2 Go) donc assez gourmand en m√©moire, mais qui est exactement celui utilis√© pour C√©mantix.
Veuillez placer le mod√®le choisi dans le dossier `gensim-data` de votre dossier utilisateur.

In [2]:
# T√©l√©chargement, si n√©cessaire.
# import wget
# url = 'https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin'
# wget.download(url, path_to_model)

In [3]:
from gensim.models import KeyedVectors

In [4]:
path_to_model = "C:\\Users\\Julien\\Documents\\HEIG-VD\\S5_2024\\APN\\APN\\lab4\\frWac_no_postag_phrase_500_cbow_cut10.bin" # √† adapter

In [5]:
model = KeyedVectors.load_word2vec_format(path_to_model, binary=True, unicode_errors="ignore")

a. Choisissez deux mots de sens proches `w1` et `w2`, et un autre plus diff√©rent not√© `w3`. Affichez la similarit√© selon word2vec (cosinus) entre chaque paire de mots.  Ces valeurs correspondent-elles √† vos intuitions ?

In [6]:
w1 = 'chat'
w2 = 'f√©lin'
w3 = 'lapper'

sim_w1_w2 = model.similarity(w1, w2)
sim_w1_w3 = model.similarity(w1, w3)
sim_w2_w3 = model.similarity(w2, w3)

print(f"Similarit√© entre '{w1}' et '{w2}': {sim_w1_w2:.4f}")
print(f"Similarit√© entre '{w1}' et '{w3}': {sim_w1_w3:.4f}")
print(f"Similarit√© entre '{w2}' et '{w3}': {sim_w2_w3:.4f}")


Similarit√© entre 'chat' et 'f√©lin': 0.5748
Similarit√© entre 'chat' et 'lapper': 0.1546
Similarit√© entre 'f√©lin' et 'lapper': 0.1847


b. Affichez les 10 mots les plus proches de `w1` selon word2vec avec pour chacun sa similarit√© avec `w1`. 

In [7]:
w1 = 'chat'
#mots voisins
neighbour_words = model.most_similar(w1, topn=10)
print(f"Les 10 mots les plus proches de '{w1}")

for word, similarity in neighbour_words:
    print(f"{word}: {similarity:.4f}")

Les 10 mots les plus proches de 'chat
minou: 0.6335
matou: 0.6266
petit_chatte: 0.6141
chatte: 0.5922
miauler: 0.5909
chaton: 0.5889
quoi_fouetter: 0.5832
f√©lin: 0.5748
chien: 0.5650
chats: 0.5555


c. Quels sont les 15 premiers coefficients du vecteur (_embedding_) du mot `w1` ? Quelle est la dimension de ce vecteur ?  Quelle est la taille du vocabulaire connu du mod√®le ?  Vous pouvez simplement √©crire les commandes r√©pondant √† ces questions.

In [8]:
w1 = 'chat'
print(f"Les 15 premiers coefficients pour '{w1}':")
print(model[w1][:15])

print(f"La dimension du vecteur'{w1}': {len(model[w1])}")

print(f"Taille du vocabulaire du mod√®le : {len(model.index_to_key)}")


Les 15 premiers coefficients pour 'chat':
[-0.5510563  -0.22379288  0.72409904 -0.82713395 -0.68234926  2.0828469
 -0.30507645  0.400353    0.9901076  -1.2962811   1.2319442   1.5554144
  0.3478864  -1.6153904  -0.585831  ]
La dimension du vecteur'chat': 500
Taille du vocabulaire du mod√®le : 1081995


d. Veuillez √©crire une fonction appel√©e `neighbors` selon les sp√©cifications suivantes.  Cette fonction servira plus loin pour l'affichage.
* input : mod√®le, liste de mots, nombre de voisins (`topn`) ;
* output : liste de mots voisins de chacun des mots de l'input, repr√©sent√©s par un dictionnaire expliqu√© ci-apr√®s ;
* fonctionnement : pour _chacun_ des mots donn√©s en input, la fonction teste si le mot est dans le vocabulaire du mod√®le word2vec (sinon elle ne le consid√®re pas), puis demande au mod√®le la liste des `topn` mots voisins ; pour chacun de ces mots, la fonction construit un `dict` √† 4 champs : 
  - mot voisin
  - similarit√©
  - mot de d√©part
  - code de couleur associ√© au mot de d√©part : 1, 2, etc.

**Exemple** : si on appelle la fonction avec \['√©cole'\], le d√©but du r√©sultat sera :
```
[{'neighbor': 'scolaire',
  'similarity': 0.7114928364753723,
  'ref_word': '√©cole',
  'color_code': 1}, ...
```

In [9]:
def neighbors(model, word_list, topn = 5):
    results = []
    for idx, word in enumerate(word_list):
        if word in model:

            similar_words = model.most_similar(word, topn=topn)
            
            for neighbor, similarity in similar_words:
                results.append({
                    'neighbor': neighbor,
                    'similarity': similarity,
                    'ref_word': word,
                    'color_code': idx + 1})
        else:
            print(f"Le mot '{word}' n'est pas dans le vocabulaire du mod√®le.")
    
    return results

In [10]:
# Tester avec cet appel :
print(neighbors(model, ['chat', '√©tudier'], 3))

[{'neighbor': 'minou', 'similarity': 0.6334686875343323, 'ref_word': 'chat', 'color_code': 1}, {'neighbor': 'matou', 'similarity': 0.626587450504303, 'ref_word': 'chat', 'color_code': 1}, {'neighbor': 'petit_chatte', 'similarity': 0.6141418218612671, 'ref_word': 'chat', 'color_code': 1}, {'neighbor': '√©tude', 'similarity': 0.6593688726425171, 'ref_word': '√©tudier', 'color_code': 2}, {'neighbor': 'analyser', 'similarity': 0.6340749263763428, 'ref_word': '√©tudier', 'color_code': 2}, {'neighbor': 'approfondir', 'similarity': 0.5609511733055115, 'ref_word': '√©tudier', 'color_code': 2}]


## 2. Affichage simple des voisins d'un mot

Pour commencer, veuillez √©tudier et ex√©cuter le code fourni en exemple : `Labo4_plotly.ipynb`.  Cela vous montrera comment utiliser l'affichage 2D/3D avec `plotly.js`, et vous permettra de traiter les questions suivantes.

Veuillez √©crire et ex√©cuter une fonction qui effectue les op√©rations suivantes, √©tant donn√© un mot en entr√©e :
* obtenir les vecteurs word2vec du mot et de ses `topn` plus proches voisins ;
* transformer ces vecteurs en 2D par l'ACP ;
* afficher en 2D chaque points correspondant √† un mot, avec comme √©tiquette le mot lui-m√™me.

**Notes:** cette fonction sert de version pr√©liminaire √† une fonction plus g√©n√©rique demand√©e ci-dessous.  Il n'est pas demand√© de distinguer le mot entr√© des mots voisins, dans l'affichage (m√™me type de points).  Il n'est pas n√©cessaire pour l'instant d'utiliser la fonction `neighbors`.

In [11]:
import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

In [12]:
def display_PCA_2D_neighbors(model, word, topn = 5):
    if word not in model:
        print(f"Le mot '{word}' n'est pas dans le vocabulaire du mod√®le.")
        return
    
    similar_words = model.most_similar(word, topn=topn)
    words = [word] + [w[0] for w in similar_words]  # Liste de mots incluant le mot de d√©part
    vectors = np.array([model[w] for w in words])   # R√©cup√©rer les vecteurs correspondants
    
    pca = PCA(n_components=2)
    vectors_2d = pca.fit_transform(vectors)
    
    trace = go.Scatter(
        x=vectors_2d[:, 0],
        y=vectors_2d[:, 1],
        mode='markers+text',
        text=words,
        textposition='top center',
        marker=dict(size=10, color='blue')
    )
    layout = go.Layout(
        title=f"Projection 2D de '{word}' et de ses voisins",
        xaxis=dict(title='CP 1'),
        yaxis=dict(title='CP 2'),
        showlegend=False
    )

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()

In [13]:
display_PCA_2D_neighbors(model, 'chat', 20)

## 3. Affichage configurable des voisins de plusieurs mots

Le but maintenant est d'√©crire une fonction `display_dimred_neighbors` qui affiche sur un seul graphique les mots voisins de chaque mot d'une liste de mots donn√©s.  Vous utiliserez ici la fonction auxiliaire `neighbors` d√©finie plus haut  Les param√®tres suivants seront pass√©s √† la fonction `display_dimred_neighbors` :

* `model` - le nom du mod√®le word2vec
* `word_list` - liste de mots dont on veut afficher les voisins (s'ils existent dans `model`)
* `n_components` - dimensionnalit√© de l'affichage, 2 ou 3
* `topn` - nombre de voisins √† afficher pour chaque mot 
* `method` - **m√©thode de r√©duction de dimensionnalit√© : pca, mds, isomap, tsne, ou umap**
* `n_neighbors`- nombre de voisins consid√©r√©s par la m√©thode (applicable √† isomap, umap, et tsne (appel√© alors `perplexity`)).

Trois tests sont demand√©s √† la question 4, mais vous pouvez en ex√©cuter d'autres et les inclure ici.

In [14]:
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from umap import UMAP # Attention, installer le module nomm√© 'umap-learn' (avec conda install).

In [62]:
def display_dimred_neighbors(model, word_list, n_components = 3, topn = 5, method = 'pca', n_neighbors = 5):
    # Obtenir les voisins pour chaque mot √† partir de la fonction `neighbors`
    all_neighbors = neighbors(model, word_list, topn=topn)
    
    # V√©rifier si des voisins ont √©t√© trouv√©s
    if not all_neighbors:
        print("Aucun mot voisin n'a √©t√© trouv√©.")
        return
    
    # R√©cup√©rer les vecteurs et les √©tiquettes
    words = [entry['neighbor'] for entry in all_neighbors] + [entry['ref_word'] for entry in all_neighbors]
    unique_words = list(set(words))  # Eliminer les doublons
    vectors = np.array([model[word] for word in unique_words])

    
    # Choisir la m√©thode de r√©duction de dimensionnalit√©
    if method == 'pca':
        reducer = PCA(n_components=n_components)
    elif method == 'mds':
        reducer = MDS(n_components=n_components, n_init=4, max_iter=300)
    elif method == 'isomap':
        reducer = Isomap(n_components=n_components, n_neighbors=n_neighbors)
    elif method == 'tsne':
        reducer = TSNE(n_components=n_components, perplexity=n_neighbors, n_iter=500, random_state=42)
    elif method == 'umap':
        reducer = UMAP(n_components=n_components, n_neighbors=n_neighbors, random_state=42)
    else:
        raise ValueError("M√©thode de r√©duction de dimensionnalit√© non reconnue.")
    
    # R√©duction des vecteurs en n dimensions
    vectors_reduced = reducer.fit_transform(vectors)
    
    # Cr√©ation des traces pour l'affichage
    if n_components == 2:
        trace = go.Scatter(
            x=vectors_reduced[:, 0],
            y=vectors_reduced[:, 1],
            mode='markers+text',
            text=unique_words,
            marker=dict(size=10, color='rgba(93, 164, 214, 0.8)'),
            textposition='top center'
        )
    elif n_components == 3:
        trace = go.Scatter3d(
            x=vectors_reduced[:, 0],
            y=vectors_reduced[:, 1],
            z=vectors_reduced[:, 2],
            mode='markers+text',
            text=unique_words,
            marker=dict(size=5, color='rgba(255, 0, 0, 0.8)'),
            textposition='top center'
        )
    
    # D√©finir le layout
    layout = go.Layout(
        title=f"Projection {n_components}D des voisins de {', '.join(word_list)} par {method.upper()}",
        width=1200,
        height=1000,
        scene=dict(
            xaxis=dict(title='Composante 1'),
            yaxis=dict(title='Composante 2'),
            zaxis=dict(title='Composante 3') if n_components == 3 else None
        ),
        xaxis=dict(title='Composante 1') if n_components == 2 else None,
        yaxis=dict(title='Composante 2') if n_components == 2 else None,
        showlegend=False
    )
    
    # Cr√©er et afficher la figure
    fig = go.Figure(data=[trace], layout=layout)
    fig.show()
    return unique_words

# Exemple d'utilisation
display_dimred_neighbors(model, ['chat', 'chien'], n_components=2, topn=5, method='umap')

c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



['chien',
 'minou',
 'petit_chatte',
 'aboyer',
 'chat',
 'toutou',
 'chiens',
 'matou',
 'cocker',
 'chienne',
 'chatte',
 'miauler']

## 4. Application √† l'√©tude des mots voisins dans word2vec

Veuillez choisir quatre mots (`word_list = [m1, m2, m3, m4]`) de fa√ßon √† ce que m1 et m2 soient tr√®s proches par leur sens ou leur usage, m3 un peu plus √©loign√©, et m4 tr√®s √©loign√©.  L'objectif de cette question est d'√©tudier la distribution des mots voisins de ces quatre mots (entre 10 et 50) par une visualisation en 2D ou en 3D des vecteurs de mots.  

En exp√©rimentant avec plusieurs configurations (m√©thode de r√©duction de dimensionnalit√©, valeur de `n_neighbors`, nombre de voisins `topn`), veuillez r√©pondre aux questions suivantes, √† l'aide d'un ou plusieurs graphiques par question :

**a.** si on utilise une m√©thode de r√©duction de dimensionnalit√© lin√©aire (PCA ou MDS m√©trique), les quatre groupes de mots sont-ils clairement s√©par√©s dans word2vec ?

**b.** si on utilise une m√©thode de r√©duction de dimensionnalit√© non-lin√©aire (Isomap, t-SNE ou UMAP), peut-on mieux mettre en √©vidence les quatre groupes de mots ?  Quels sont les meilleurs param√®tres que vous avez trouv√©s permettant de bien identifier les quatre ensembles ?

**c.** inversement, trouvez-vous des param√®tres qui aboutissent √† cinq clusters ? ou √† trois ?

In [43]:
# Ins√©rer ici la liste de mots.
mots = ['grecque', 'latin', 'oublier', 'nourriture']

In [148]:
# Question 4a
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='pca', n_neighbors=10)
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='mds', n_neighbors=10)
#
# En utilisant la m√©thode PCA, on peut voir que les mots 'latin' et 'grecque' sont proches l'un de l'autre (meme cluster)
# Nous observons √©galement un cluster diff√©rent pour 'nourriture' et 'oublier'
# En revanche pour la m√©thode MDS, les mots 'latin' et 'grecque' sont plus √©loign√©s l'un de l'autre ainsi que la densit√© en plus homog√®ne, les clusters ne sont plus √©vidents de mani√®re visuelle.
# A noter que la position des diff√©rents cluster reste la m√™me pour les deux m√©thodes.



['dire',
 'nouriture',
 'l√†',
 'nourrir',
 'jamais',
 'hell√®nes',
 'ration_quotidien',
 'ne',
 'gr√©co-',
 'latin_m√©di√©val',
 'greque',
 've_si√®cle_av_j√©sus-christ',
 'ionien',
 'grecquer',
 'falloir',
 'oublier',
 'vivres',
 'grecs',
 'hell√©nique',
 'iiie_si√®cle_av_j√©sus-christ',
 'mot_latin',
 'proie_vivant',
 'latiniser',
 'p√¢t√©e',
 'langue_vulgaire',
 'rien',
 '√©tymologie',
 'racine_grec',
 'grammairien',
 'grec',
 'verbe_latin',
 'affamer',
 'nourriture',
 'aliment',
 'mais',
 'nourriture_sain',
 'manger',
 'latiniste',
 'pas',
 'latin',
 'toujours',
 'grecque',
 'quand']

In [ ]:
# Question 4b
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='tsne', n_neighbors=3)
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='isomap', n_neighbors=3)
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='umap', n_neighbors=3)
# Avec la m√©thode TSNE, on observe que les clusters sont tr√®s proches les uns des autres, m√™me les mots dont la distance s√©mantique est grande.
# Avec la m√©thode Isomap, on observe que les clusters sont plus √©loign√©s les uns des autres, mais les mots 'latin' et 'grecque' sont plus proches l'un de l'autre. Nous observons
# √©galement un 'point' de contact entre les 3 mots.
# Avec la m√©thode UMAP, on observe que les clusters sont plus √©loign√©s les uns des autres, nous obtenons de vrais clusters distincts pour chaque mot.

c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\sklearn\manifold\_t_sne.py:1162: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



['dire',
 'nouriture',
 'l√†',
 'nourrir',
 'jamais',
 'hell√®nes',
 'ration_quotidien',
 'ne',
 'gr√©co-',
 'latin_m√©di√©val',
 'greque',
 've_si√®cle_av_j√©sus-christ',
 'ionien',
 'grecquer',
 'falloir',
 'oublier',
 'vivres',
 'grecs',
 'hell√©nique',
 'iiie_si√®cle_av_j√©sus-christ',
 'mot_latin',
 'proie_vivant',
 'latiniser',
 'p√¢t√©e',
 'langue_vulgaire',
 'rien',
 '√©tymologie',
 'racine_grec',
 'grammairien',
 'grec',
 'verbe_latin',
 'affamer',
 'nourriture',
 'aliment',
 'mais',
 'nourriture_sain',
 'manger',
 'latiniste',
 'pas',
 'latin',
 'toujours',
 'grecque',
 'quand']

In [ ]:
# Question 4c
display_dimred_neighbors(model, mots, n_components=3, topn=20, method='umap', n_neighbors=10)
# obtenons un cluster plus grand pour les mots 'latin' et 'grecque' avec cette m√©thode. Nous pourrions consid√©rer que celui ci forme deux sous cluster distincts.
# Par exemple des noms d'auteurs et des concepts grammaticaux
# 

c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



['dire',
 'excr√©ment',
 'l√†',
 'nourrir',
 'jamais',
 'lui',
 'r√©gime_alimentaire',
 'ne',
 'vouloir',
 'latin_m√©di√©val',
 've_si√®cle_av_j√©sus-christ',
 'ionien',
 'achille_tatius',
 'grecquer',
 'proie_vivant',
 'iiie_si√®cle_av_j√©sus-christ',
 'hell√©nistique',
 'latiniser',
 'p√¢t√©e',
 '√©poque_myc√©nien',
 'etymologie',
 'affamer',
 'verbe_latin',
 'aliment',
 'car',
 'latiniste',
 'origine_germanique',
 'toujours',
 'nouriture',
 'penser',
 'ration_quotidien',
 'gr√©co-',
 'peut-√™tre',
 'caput',
 'nourriture_abondant',
 'racine_indo-_europ√©en',
 'vivres',
 'hell√©nique',
 'nourriture',
 'h√©racl√©e',
 'mais',
 'pas',
 'latin',
 'grecques',
 'perses',
 'quand',
 'juste',
 'greque',
 'victuaille',
 'nourrir_exclusivement',
 'tellement',
 'rien',
 '√©tymologie',
 'racine_grec',
 'grec',
 'dorien',
 'croire',
 'heureusement',
 'frugal',
 'po√®me_hom√©rique',
 'sinon',
 'grecque',
 'm√©nandre',
 'hell√®nes',
 'clitophon',
 'ration_alimentaire',
 'denr√©e',
 'gr√©co-_latin',


## 5. Outil d'assistance pour le jeu C√©mantix

Le jeu en ligne [C√©mantix](https://cemantix.certitudes.org/) (aussi propos√© sur le site [Dictaly](https://www.dictaly.com/semantiques/))  demande de deviner le *mot du jour* en l'approchant peu √† peu par des mots candidats.  Pour chacun, le syst√®me indique pour sa similarit√© word2vec avec le *mot du jour*, ce qui permet de se rapprocher graduellement de la solution.  Exp√©rimentez d'abord un court instant avec le jeu üòÄ.

L'objectif de cette question est de visualiser les voisinages de mots, pour vous aider √† proposer des mots candidats et trouver plus vite la solution.  La proc√©dure est la suivante :
* essayez trois mots au hasard dans C√©mantix
* affichez 20-30 mots voisins de ces trois mots gr√¢ce √† la fonction `display_dimred_neighbors`
* √† l'aide des mots affich√©s, essayez des mots candidats dans C√©mantix
* changez l'affichage en rempla√ßant les trois mots par de meilleurs mots
* continuez jusqu'√† trouver le *mot du jour*

**Notes**
* si vous le souhaitez, vous pouvez utiliser le mod√®le word2vec identique √† celui de C√©mantix, qui est frWac_no_postag_phrase_500_cbow_cut10.bin fourni par [J.-Ph. Fauconnier](https://fauconnier.github.io/#data)
* ce mod√®le fait environ 2.3 Go et contient aussi des expressions de plusieurs mots (s√©par√©s par '_')
* vous pouvez choisir d'utiliser le petit ou le grand mod√®le, sachant qu'avec le premier, les suggestions sont moins pertinentes
* C√©mantix ignore les expressions de plusieurs mots pr√©sentes dans le grand mod√®le -- vous pouvez choisir de le faire ou non.

In [165]:
# candidate_words = ['homme', 'rouge', '√©tudier']
candidate_words = ['salle', 'conf√©rence', 'espace']
m = display_dimred_neighbors(model, candidate_words, n_components=3, topn=20, method='pca', n_neighbors=5)


In [166]:
'atelier' in m
# afficher les voisin de atelier
m

['salles',
 'spatial',
 'paysage',
 'rez-de-chauss√©e',
 'r√©fectoire',
 'conf√©rencier',
 'salle_attenant',
 'lieux',
 'conf√©rence',
 'urbain',
 'amphith√©√¢tre',
 'privil√©gier',
 'er_√©tage',
 'table-ronde',
 'vestiaire_douche',
 'conf√©rence_pl√©nier',
 's√©minaire',
 'dimension',
 'conference',
 'espaces',
 'hall',
 'espace_interstitiel',
 'culturel',
 'coorganiser',
 'conf√©rence_d√©battre',
 'organiser_conjointement',
 'espace',
 'gymnase',
 's√©ance_inaugural',
 '√©l√©ment_structurant',
 'caf√©t√©ria',
 'spatialit√©',
 'spatialement',
 'dimension_spatial',
 'colloque',
 'table_rond',
 'salle_modulable',
 'symposium',
 'am√©nager',
 'espace_privatif',
 'espace_sc√©nique',
 'paysager',
 'conf√©rence-d√©bat',
 'rotonde',
 'trame_urbain',
 'salle',
 '√©tage',
 'conf√©rencier_inviter',
 'cheminement_pi√©ton',
 'conferences',
 'tables-ronde',
 'table_chaise',
 'locaux',
 'conf√©rences',
 'conf√©rence_inaugural',
 'vaste_hall',
 'cafeteria',
 'tissu_urbain',
 'conf√©rences-d√©bat',
 

Veuillez noter ici vos observations sur la proc√©dure, et coller un extrait montrant votre meilleure performance au jeu.

La visualisation des voisions en pca nous aide √† d√©marrer le jeu avec tr√®s peu de mots d√©couverts.

![image.png](c.jpg)

**Fin du Labo.**  Veuillez nettoyer ce notebook en gardant seulement les r√©sultats d√©sir√©s, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.